In [ ]:
# Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
# Import the data
df_train = pd.read_csv('Resources/TitanicData.csv')
df_train.head()

In [ ]:
# Removing unwanted columns
df_train.drop(["Name","PassengerId","Ticket","Cabin"], axis=1, inplace=True)
df_train.head()

In [ ]:
# Adjusting Pclass column to strings
df_train['Pclass'] = df_train['Pclass'].replace([1,2,3],['1st','2nd','3rd'])
df_train.head()

In [ ]:
# Dropping NAN Values
df_train = df_train.dropna(how='any')

In [ ]:
# Converting categorical data to dummy values for testing
df_train = pd.get_dummies(df_train)

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
y = df_train["Survived"].values
X = df_train.drop("Survived", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.head()

In [ ]:
# Scaling Data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train a Logistic Regression model print the model score
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='lbfgs',max_iter=1000)
classifier.fit(X_train, y_train)
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")